In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm

In [2]:
Data_path = "../../Data/"
file_all_pupil = os.listdir(Data_path+"PupilData/")
len(file_all_pupil)

108

In [3]:
def timestamp_to_utc_datetime(timestamp):
 
    """将 13 位整数的毫秒时间戳转化成 utc 时间 (datetime 格式)
    :param timestamp: 13 位整数的时间戳 (1456402864242)
    :return: 返回 datetime 格式 {datetime}2016-02-25 12:21:04.242000
    """

    utc_dt_time = datetime.utcfromtimestamp(timestamp / 1000.0)
    

    return utc_dt_time

In [4]:
def data_befo_after(path_pupil,path_event):
    """
    根据测试Event的时间对瞳孔数据进行删减，去除实验前后的多余数据
    
    """
    
    file_new_event = pd.read_csv(path_event)
    file_new_pupil = pd.read_csv(path_pupil)
#     print(len(file_new_event))
#     print(len(file_new_pupil))
    file_new_event['timestamp'] = pd.to_datetime(file_new_event['timestamp'])
    file_new_pupil['timestamp'] = pd.to_datetime(file_new_pupil['timestamp'])
    # 删除实验开始前的数据
    # Delete PupilData before Experiment Event
    file_new_pupil = file_new_pupil.drop(file_new_pupil[file_new_pupil['timestamp']<file_new_event['timestamp'][0]].index[:-1])
    # 删除实验后的数据
    # Delete PupilData After Experiment Event
    file_new_pupil = file_new_pupil.drop(file_new_pupil[file_new_pupil['timestamp']>file_new_event['timestamp'][len(file_new_event['timestamp'])-1]].index)
    # 保存到单独的文件夹
    # Save to a separate folder
    file_new_pupil.to_csv(Data_path+"/processed/"+patient_name+"/"+"sub"+patient_name+".dat",index=False)
#     # 保存到组间分析的文件夹
#     file_new_pupil.to_csv("分析结果/"+"sub"+patient_name+".dat",index=False)
    

In [5]:
def event_fun(patient_name):
    """
    对event数据进行操作
    1. 删除不需要的特征
    2. 提取timestamp列表
    3. 提取message列表
    4. 保存结果
    """

    file_event = pd.read_csv(Data_path+"/EventData/"+patient_name+".csv")
    file_event = file_event[['timestamp_stake', 'timestamp_stakeDisappear', 'timestamp_spaceship','timestamp_spaceshipResponse', 'timestamp_monster','timestamp_monsterResponse', 'timestamp_reward','transitions','stake']]
    #
    # 删除不符合要求的数据
    #
    for i in ['timestamp_stake', 'timestamp_stakeDisappear', 'timestamp_spaceship','timestamp_spaceshipResponse', 'timestamp_monster','timestamp_monsterResponse', 'timestamp_reward']:
        file_event = file_event.drop(file_event[file_event[i] == 0].index)
    file_event = file_event.reset_index(drop=True)
    file_event["timestamp_end_wang"] = file_event["timestamp_stake"]+9000
    order = ['timestamp_stake', 'timestamp_stakeDisappear', 'timestamp_spaceship',
       'timestamp_spaceshipResponse', 'timestamp_monster',
       'timestamp_monsterResponse', 'timestamp_reward', 'timestamp_end_wang','transitions', 'stake']
    file_event = file_event[order]
    #
    # 创建timestamp列
    #
    for i in file_event.columns[:-2]:
        file_event[i] = file_event[i].apply(timestamp_to_utc_datetime)
    file_event
    file_new_event = pd.DataFrame(columns=["timestamp","message"])
    file_new_event["timestamp"] = file_event.values[:,:-2].flatten()
    #
    # 创建message列
    #
    temp = []
    for index in range(len(file_new_event)//8):
        temp.append("TRIALID %d"%(index+1))
        temp.append("!V TRIAL_VAR transitions %s"%file_event['transitions'][index])
        temp.append("!V TRIAL_VAR stake %s"%file_event['stake'][index])
        temp.extend(["!E TRIAL_EVENT_VAR response_1","!E TRIAL_EVENT_VAR monster_show","!E TRIAL_EVENT_VAR response_2","!E TRIAL_EVENT_VAR reward","TRIAL_END"])
    file_new_event["message"] = temp
    # 保存到单独的文件夹
    file_new_event.to_csv(Data_path+"/processed/"+patient_name+"/sub"+patient_name+"_events.csv",index=False)
#     # 保存到组间分析文件夹
#     file_new_event.to_csv("./分析结果/"+"/sub"+patient_name+"_events.csv",index=False)
    # 删除非试验期间数据
    data_befo_after(Data_path+"/processed/"+patient_name+"/"+"sub"+patient_name+".dat",Data_path+"/processed/"+patient_name+"/sub"+patient_name+"_events.csv")
    

In [6]:
# 没有对应的游戏数据,将被判定为无效数据
temp_no_event_data = []
for i in tqdm(file_all_pupil):
    import time
    # 读取一个瞳孔文件
    # read PupilData
    file_pupil = pd.read_csv(Data_path+"/PupilData/"+i)
    # 提取被试名称
    # extract name
    patient_name = i.split("-")[0]+"-"+i.split("-")[1]
    # file_event_name = i.split("-")[0]+"-"+i.split("-")[1]+".csv"
    file_new_pupil = pd.DataFrame(columns=["timestamp","pupil_size","pupil_x","pupil_y"])
    
    file_new_pupil["timestamp"] = file_pupil["timestamp_ms"].apply(timestamp_to_utc_datetime)
    file_new_pupil["pupil_size"] = file_pupil["diameter_px"]
    file_new_pupil["pupil_x"] = file_pupil["center_x"]
    file_new_pupil["pupil_y"] = file_pupil["center_y"]
#     print(Data_path+"/processed/"+patient_name+"/"+"sub"+patient_name+".dat")
#     if os.path.exists(Data_path+"/processed/"+patient_name+"/"):
#         print("存在")
#     else:
#         print("不存在")
#         os.mkdir(Data_path+"/processed/"+patient_name+"/")
    if not os.path.exists(Data_path+"/processed/"):
        os.mkdir(Data_path+"/processed/")
    if not os.path.exists(Data_path+"/processed/InvalidData"):
        os.mkdir(Data_path+"/processed/InvalidData")
    if os.path.exists(Data_path+"/EventData/"+patient_name+".csv"):
        if not os.path.exists(Data_path+"/processed/"+patient_name+"/"):
            os.mkdir(Data_path+"/processed/"+patient_name+"/")
        file_new_pupil.to_csv(Data_path+"/processed/"+patient_name+"/"+"sub"+patient_name+".dat",index=False)
        event_fun(patient_name)    
    else:
#         print(patient_name+"对应的文件不存在")
#         print(patient_name+"The corresponding Event data does not exist")
        file_new_pupil.to_csv(Data_path+"/processed/InvalidData/"+"sub"+patient_name+".dat",index=False)
        
        temp_no_event_data.append(patient_name)

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [09:38<00:00,  5.36s/it]


In [7]:
print("数据总共"+str(len(file_all_pupil))+"无效率"+str(round(len(temp_no_event_data)/len(file_all_pupil)*100,1))+"%")

数据总共108无效率15.7%


In [9]:
pwd

'D:\\Desktop\\百度网盘同步\\BaiduNetdiskWorkspace\\组队学习\\开源代码\\瞳孔数据处理\\正式开源\\jupyterlab\\数据准备'